In [1]:
import argostranslate.package
import argostranslate.translate
from langdetect import detect, lang_detect_exception

In [28]:
s = ""#"नमस्ते, आप कैसे हैं"#"สวัสดี,สบายดีไหม"

print(detect(s))

print(argostranslate.translate.translate(s, 'vi', "en"))

for sym in s:
    try:
        print(sym)
        print(detect(sym))
    except lang_detect_exception.LangDetectException:
        continue



th


AttributeError: 'NoneType' object has no attribute 'get_translation'

In [49]:
def processKO(sym):
    code = ord(sym)
    if 0x3040 <= code <= 0x309F or 0x30A0 <= code <= 0x30FF:
        return "ja"
    # 韩语谚文
    if 0xAC00 <= code <= 0xD7AF or 0x1100 <= code <= 0x11FF:
        return "ko"
    # 中文（CJK 统一汉字）
    if 0x4E00 <= code <= 0x9FFF:
        return "zh-cn"

def mydetect(text):
    class Part:
        def __init__(self, text = "", lang = None, start = 0, end = 0):
            self.text = text 
            self.lang = lang 
            self.start = start 
            self.end = end

        def __str__(self):
            return "(" + ", ".join(map(str, [self.text, self.lang, self.start, self.end])) + ")"

        def __repr__(self):
            return str(self)

    parts = [Part()]
    start = 0
    # separate by parts:
    for i, sym in enumerate(text):
        try:
            detect(sym)
            parts[-1].text += sym
        except lang_detect_exception.LangDetectException:
            # this error is mostly thrown when the separation sign is detected
            if sym != " " and len(parts[-1].text):
                parts[-1].start = start
                parts[-1].end = i
                parts[-1].lang = detect(parts[-1].text)

                if parts[-1].lang == "ko":
                    parts[-1].lang = processKO(parts[-1].text[0])
                
                parts.append(Part())

            start = i + 1
            continue

    parts[-1].lang = detect(parts[-1].text)
    parts[-1].start = start
    parts[-1].end = len(text)

    if parts[-1].lang == "ko":
        parts[-1].lang = processKO(parts[-1].text[0])

    return parts

s = "软件(Software)是一系列按照, 안녕하세요, 特定(สวัสดี เป็นยังไงบ้าง)顺序组织的電腦数据和指令，是電腦中的非有形部分。電腦中的有形部分稱為硬體，由電腦的外殼及各零件及電路所組成。電腦軟體需有硬體才能運作，反之亦然，軟體和硬體都無法在不互相配合的情形下進行實際的運作。"

mydetect(s)

[(软件, zh-cn, 0, 2),
 (Software, en, 3, 11),
 (是一系列按照, zh-cn, 12, 18),
 (안녕하세요, ko, 20, 25),
 (特定, zh-cn, 27, 29),
 (สวัสดีเป็นยังไงบ้าง, th, 37, 50),
 (顺序组织的電腦数据和指令，是電腦中的非有形部分。電腦中的有形部分稱為硬體，由電腦的外殼及各零件及電路所組成。電腦軟體需有硬體才能運作，反之亦然，軟體和硬體都無法在不互相配合的情形下進行實際的運作。, zh-cn, 51, 149)]

In [48]:
hex(ord("顺"))

'0x987a'